# Modules
In Python, a module must be defined in a dedicated file. In Julia, modules are independent from the file system. You can define several modules per file, or define one module across multiple files, it's up to you. Let's create a simple module containing two submodules, each containing a variable and a function:

In [5]:
module ModA
    e = ℯ
    f(x) = sin(x)
    module ModB
        p = π
        g(x) = cos(x)
        module ModC
            x = 1.0
            h(x) = log(x)
        end
    end    
    module ModD
        y = 1729 # 1^3 + 12^3 = 10^3 + 9^3
        l(x) = x^5
    end
end

Main.ModA

The default module is `Main`, so whatever we define is put in this module (except when defining a package, as we will see). This is why the `ModA`'s full name is `Main.ModA`.

We can now access the contents of these modules by providing the full paths:

In [ ]:
Main.ModA.ModB.ModC.h(ℯ)
# ModA.ModB.ModC.h(ℯ)

Since our code runs in the `Main` module, we can leave out the `Main.` part:

Alternatively, you can use `import`:

In [14]:
# import .ModA.ModB.ModC.x
import Main.ModA.ModB.ModC.x
x

1.0

Make sure that you put Main at the beginning of the command. Or we can use `import` with a relative path. In this case, we need to prefix `ModA` with a dot `.` to indicate that we want the module `ModA` located in the current module:

In [18]:
import .ModA.ModD
println(ModD.y)

1729


Nested modules do <u>not</u> automatically have access to names in enclosing modules. To import names from a parent module, use `..x`. From a grand-parent module, use `...x`, and so on.

In [19]:
module ModA2
    e = ℯ
    f(x) = sin(x)
    module ModB2
        p = π
        g(x) = cos(x)
        module ModC2
            import ..p
            import ...e
            x = 1.0
            h(x) = log(x) + p / e 
        end
    end
    module ModD2
        y = 1729 # 1^3 + 12^3 = 10^3 + 9^3
        my_l(x) = x^5
    end
end

Main.ModA2

Instead of `import`, you can use `using`. It is analog to Python's `from foo import *`. It only gives access to names which were explicitly exported using `export` (similar to the way `from foo import *` in Python only imports names listed in the module's `__all__` list):

In [20]:
import .ModA2.ModB2.ModC2
ModC2.h(exp(-π/ℯ))

0.0

In [21]:
module ModE
    z = 100
    u = 200
    w = 300
    v = 400
    export z
end

Main.ModE

Note that `using Foo` not only imports all exported names (like Python's `from foo import *`), it also imports `Foo` itself (similarly, `using Foo.Bar` imports `Bar` itself):


Even if a name is not exported, you can always access it using its full path, or using `import`:

You can also import individual names like this:

In [23]:
module ModG
    g1 = 1
    g2 = 2
    export g2
end

using .ModG: g1, g2

println(g1)
println(g2)

1
2


Notice that this syntax gives you access to any name you want, whether or not it was exported. In other words, whether a name is exported or not only affects the `using Foo` syntax.

Importantly, when you want to expand a function which is defined in a module, you must import the function using `import`, or you must specify the function's path:

In [38]:
module ModH
    double(x) = x * 2
    triple(x) = x * 3
end

import .ModH: double
double(x::AbstractString) = repeat(x, 2)

ModH.triple(x::AbstractString) = repeat(x, 3)

println(double(2))
println(double("Two"))

println(ModH.triple(3))
println(ModH.triple("Three"))

4
TwoTwo
9
ThreeThreeThree


You must never extend a function imported with `using`, unless you provide the function's path:

In [42]:
module ModI
    quadruple(x) = x * 4
    export quadruple
end

using .ModI
ModI.quadruple(x::AbstractString) = repeat(x, 4) # OK
println(quadruple(4))
println(quadruple("Four"))

# quadruple(x::AbstractString) = repeat(x, 4) # uncomment to see the error

16
FourFourFourFour


In general, a module named `Foo` will be defined in a file named `Foo.jl` (along with its submodules). However, if the module becomes too big for a single file, you can split it into multiple files and include these files in `Foo.jl` using the `include()` function.

For example, let's create three files: `Awesome.jl`, `great.jl` and `amazing/Fantastic.jl`, where:
* `Awesome.jl` defines the `Awesome` module and includes the other two files
* `great.jl` just defines a function
* `amazing/Fantastic.jl` defines the `Fantastic` submodule

In [45]:
code_awesome = """
module Awesome
include("great.jl")
include("amazing/Fantastic.jl")
end
"""

code_great = """
great() = "This is great!"
"""

code_fantastic = """
module Fantastic   
fantastic = true
end
"""

open(f->write(f, code_awesome), "Awesome.jl", "w")
open(f->write(f, code_great), "great.jl", "w")
mkdir("amazing")
open(f->write(f, code_fantastic), "amazing/Fantastic.jl", "w")

41

If we try to execute `import Awesome` now, it won't work since Julia does not search in the current directory by default. Let's change this:

In [46]:
pushfirst!(LOAD_PATH, ".")

4-element Vector{String}:
 "."
 "@"
 "@v#.#"
 "@stdlib"

Now when we import the `Awesome` module, Julia will look for a file named `Awesome.jl` in the current directory, or for `Awesome/src/Awesome.jl`, or for `Awesome.jl/src/Awesome.jl`. If it does not find any of these, it will look in the other places listed in the `LOAD_PATH` array (we will discuss this in more details in the "Package Management" section).

In [47]:
import Awesome
println(Awesome.great())
println("Is fantastic? ", Awesome.Fantastic.fantastic)

This is great!


Is fantastic? true


Let's restore the original `LOAD_PATH`:

In [48]:
popfirst!(LOAD_PATH)

"."

In short:

|Julia | Python
|------|-------
|`import Foo` | `import foo`
|`import Foo.Bar` | `from foo import bar`
|`import Foo.Bar: a, b` | `from foo.bar import a, b`
|`import Foo.Bar.a, Foo.Bar.b` | `from foo.bar import a, b`
|`import .Foo` | `import .foo`
|`import ..Foo.Bar` | `from ..foo import bar`
|`import ...Foo.Bar` | `from ...foo import bar`
|`import .Foo: a, b` | `from .foo import a, b`
||
|`using Foo` | `from foo import *; import foo`
|`using Foo.Bar` | `from foo.bar import *; from foo import bar `
|`using Foo.Bar: a, b` | `from foo.bar import a, b`

|Extending function `Foo.f()` | Result
|-----------------------------|--------
|`import Foo.f  # or Foo: f` <br />`f(x::Int64) = ...`  | OK
|`import Foo`<br />`Foo.f(x::Int64) = ...` | OK
|`using Foo`<br />`Foo.f(x::Int64) = ...` | OK
|`import Foo.f # or Foo: f`<br />`Foo.f(x::Int64) = ...` | `ERROR: Foo not defined`
|`using Foo`<br />`f(x::Int64) = ...` | `ERROR: Foo.f must be explicitly imported`
|`using Foo: f`<br />`f(x::Int64) = ...` | `ERROR: Foo.f must be explicitly imported`

# Scopes
Julia has two types of scopes: global and local.

Every module has its own global scope, independent from all other global scopes. There is no overarching global scope.

Modules, macros and types (including structs) can only be defined in a global scope.

Most code blocks, including `function`, `struct`, `for`, `while`, etc., have their own local scope. For example:

In [49]:
for q in 1:3
    println(q)
end

try
    println(q) # q is not available here
catch ex
    ex
end

1
2
3


UndefVarError(:q)

A local scope inherits from its parent scope:

In [50]:
z = 5
for i in 1:3
    w = 10
    println(i * w * z) # i and w are local, z is from the parent scope
end

50
100
150


An inner scope can assign to a variable in the parent scope, if the parent scope is not global:

In [61]:
for i in 1:3
    s = 0
    for j in 1:5
        s = j # variable s is from the parent scope
    end
    println(s)
end

5
5
5


You can force a variable to be local by using the `local` keyword:

In [62]:
for i in 1:3
    s = 0
    for j in 1:5
        local s = j # variable s is local now
    end
    println(s)
end

0
0
0


To assign to a global variable, you must declare the variable as `global` in the local scope:

In [65]:
for i in 1:3
    global t
    t = i
end

t

3

There is one exception to this rule: when executing code directly in the REPL (since Julia 1.5) or in IJulia, you do not need to declare a variable as `global` if the global variable already exists:

In [66]:
s = 0
for i in 1:3
    s = i # implicitly global s: only in REPL Julia 1.5+ or IJulia
end
s

3

In functions, assigning to a variable which is not explicitly declared as global always makes it local (even in the REPL and IJulia):

In [67]:
s, t = 1, 2 # globals

function f2()
   s = 10 * t # s is local, t is global
end

println(f2())
println(s)

20
1


Just like in Python, functions can capture variables from the enclosing scope (not from the scope the function is called from):

In [69]:
t = 1 # a global variable

f3() = t # foo() captures t from the global scope

function f4()
    t = 5 # this is a new local variable
    println(f3()) # f3() still uses t from the global scope
end

f4()

1


In [71]:
function f5()
    global t
    t = 5 # we change the global t
    println(f3()) # and this affects foo()
end

f5()

5


Closures work much like in Python:

In [74]:
function create_multiplier(n)
    function mul(x)
        x * n # variable n is captured from the parent scope
    end
end

mul2 = create_multiplier(2)
mul2(5)


# create_multiplier(2)(5)

100

In this example, the local variable `a` is initialized with the value of `a + 1`, where `a` comes from the parent scope (i.e., it's the global `a` in this case). However, `b` is initialized with the value of the local `a`, since it now hides the variable `a` from the parent scope.

Default values in function arguments also have this left-to-right scoping logic:

In [80]:
a = 1
parameter_updater(a=a+1, b=a) = println("a=$a, b=$b")
parameter_updater()
# parameter_updater(5)
# parameter_updater(5, 1)

a=5, b=1


In this example, the first argument's default value is `a + 1`, where `a` comes from the parent scope (i.e., the global `a` in this case). However, the second argument's default value is `a`, where `a` in this case is the value of the first argument (<u>not</u> the parent scope's `a`).

Note that `if` blocks and `begin` blocks do <u>not</u> have their own local scope, they just use the parent scope:

In [81]:
a = 1
if true
    a = 2 # same `a` as above
end
a

2

In [82]:
a = 1
begin
    a = 2  # same `a` as above
end
a

2